## Expired Futures
When searching for an expired asset, you will need to specify the date range based on the ExpiryDate property.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.DataPlatform.Content, 1.0.0-alpha3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Microsoft.Data.Analysis version 0.4.0

Installed package Refinitiv.DataPlatform.Content version 1.0.0-alpha3

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.DataPlatform.Content.SearchService;
using Refinitiv.DataPlatform.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Linq;
using System;

#### DataFrame Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<IList<JObject>>((hits, writer) =>
{
    if (hits?.Count > 0)
    {
        var headers = new List<IHtmlContent>();
        var columns = hits[0].Properties();
        headers.AddRange(columns.Select(c => (IHtmlContent) th(c.Name)));
        var rows = new List<List<IHtmlContent>>();

        var take = 20;
        for (var i=0; i < Math.Min(take, hits.Count); i++)
        {
            var cells = new List<IHtmlContent>();
            foreach (var val in hits[i].Values())
                cells.Add(td(val.ToString()));
            rows.Add(cells);
        }

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [5]:
// Create a session into the desktop
var session = CoreFactory.CreateSession(new DesktopSession.Params()
                            .AppKey("Your API Key here")
                            .OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                            .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})")));
session.Open();

13/12/2020 9:34:36 AM:Session is Pending. (State: Pending)
13/12/2020 9:34:37 AM:{
  "Contents": "Desktop Session Successfully Authenticated"
}. (Event: SessionAuthenticationSuccess)
13/12/2020 9:34:37 AM:Session is Opened. (State: Opened)


#### Example - Expired Vodafone futures in the month of Nov 2019.

In [6]:
var filter = "TickerSymbol eq 'vod' and ExpiryDate lt 2019-12-01 and ExpiryDate ge 2019-11-01";
var select = "DTSubjectName, ExchangeName, RIC, RCSAssetCategoryLeaf, AssetState, ExpiryDate, UnderlyingQuoteRIC";
var response = Search.Definition(Search.View.DerivativeQuotes).Filter(filter)
                                                              .Select(select)
                                                              .GetData();
response.Data.Hits

Johannesburg Stock Exchange Vodacom Group SSF Equity Future Nov 2019,Johannesburg Stock Exchange,VODQX9^1,Equity Future,DC,21/11/2019 12:00:00 AM,"[ ""VODJ.J"" ]"
Johannesburg Stock Exchange Vodacom Group Limited Cash Settled SSF Equity Future Nov 2019,Johannesburg Stock Exchange,VODSX9^1,Equity Future,DC,21/11/2019 12:00:00 AM,"[ ""VODJ.J"" ]"


#### Example - Expired Crude Oil Futures in the last year

In [7]:
// Build expiry date range expression
var dateOneYearAgo = DateTime.Now.AddYears(-1);
var dateRange = $"ExpiryDate ge {dateOneYearAgo:yyyy-MM-dd} and ExpiryDate lt {DateTime.Now:yyyy-MM-dd}";

In [8]:
filter = $@"RCSUnderlyingProductName xeq 'Energy' and
            SearchAllCategoryv3 eq 'Commodities' and
            RCSAssetCategoryName eq 'Derivative\Future' and RCSUnderlyingProductName eq 'Crude Oil' and
            ExchangeName xeq 'Source is a Refinitiv Contributor' and {dateRange}";
response = Search.Definition(Search.View.SearchAll).Filter(filter)
                                                   .OrderBy("ExpiryDate desc")
                                                   .Top(30)
                                                   .Select("DTSubjectName, RIC, ExpiryDate, RCSUnderlyingProductLeaf")
                                                   .GetData();
response.Data.Hits

NYMEX WTI Light Sweet Crude Oil Calendar Swap US Monthly Dec 2020,CLCALMZ0,30/11/2020 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Monthly Nov 2020,CLCALMX0,31/10/2020 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Monthly Oct 2020,CLCALMV0,30/09/2020 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Quarterly Dec 2020,CLCALQZ0,30/09/2020 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Monthly Sep 2020,CLCALMU0,31/08/2020 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Monthly Aug 2020,CLCALMQ0,31/07/2020 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Monthly Jul 2020,CLCALMN0,30/06/2020 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Quarterly Sep 2020,CLCALQU0,30/06/2020 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Monthly Jun 2020,CLCALMM0,31/05/2020 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Monthly May 2020,CLCALMK0,30/04/2020 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Monthly Apr 2020,CLCALMJ0,31/03/2020 12:00:00 AM,Crude Oil


#### Example - Expired Electricity Futures in the last year

In [25]:
filter = $@"SearchAllCategoryv3 eq 'Commodities' and
            RCSUnderlyingProductName eq 'Power' and
            ExchangeName xeq 'Source is a Refinitiv Contributor' and {dateRange}";
response = Search.Definition(Search.View.SearchAll).Filter(filter)
                                                   .OrderBy("ExpiryDate desc")
                                                   .Top(30)
                                                   .Select("DTSubjectName, RIC, ExpiryDate, RCSUnderlyingProductLeaf")
                                                   .GetData();
response.Data.Hits

TRC France Baseload Power Fair Value Day Ahead 2,TRFRBFVDc2,12/12/2020 12:00:00 AM,Baseload
Germany vs Austria Baseload Power Fair Value Spread Balance of Month (BOM) Jan 2020,TRDEATBFVBMF0,31/01/2020 12:00:00 AM,Baseload
TRPC Electricity Great Britain Peakload Monthly Jan 2020,TRGBPMF0,27/12/2019 12:00:00 AM,Peakload
